# Yatzy RL environment

Yatzy program to play with and get policy and probabilities

**Features**

* single player setting to start with
* encoding state
* transition function for rules
* checking terminal state and computing reward
* actions to roll dice and select categories
* value iteration to compute value function / policy
* scorecard - which categories are used
* check if category can be used

**Getting probabilities**

Value iteration framework naturally gives us expected values (average scores), but if we want the full probability distribution of outcomes, we have a couple of options:

* Extend our existing transition function - Since we're already building transitions (state → action → next state), we can track probabilities through these transitions. This is essentially using the Markov chain structure we're already creating.
* Monte Carlo simulation - Once we have our optimal policy, we can run many simulations and collect the distribution of final scores empirically.

The first approach is more exact but computationally intensive. The second is simpler to implement and often sufficient.

I've decided on following **state representation**

- **Current dice values** - a sorted tuple (1, 2, 3, 3, 5)
- **Rolls left** - int - determines whether we can reroll or must score
- **Filled categories** - frozenset of strings - determines which scoring options are still available

**Value function** could be a dict:
```python
# V = {state: value}
state = (dice, rolls, categories)
value = V[state]
```

**Plan to Start:**

1. Define the 15 category names (as strings for frozenset)
2. Write scoring functions for each category
3. Implement dice rolling and state transitions
4. Define valid actions (which dice to keep, which category to score)
5. Build value iteration algorithm
6. Test and iterate

In [ ]:
#| default_exp yatzy

In [ ]:
#| export
import random
from dataclasses import dataclass
from typing import Literal
import itertools
from collections import Counter
from functools import partial

In [ ]:
#| export
# categories

categories = frozenset([
    "ones",
    "twos",
    "threes",
    "fours",
    "fives",
    "sixes",
    "one_pair",
    "two_pairs",
    "three_same",
    "four_same",
    "small_straight",
    "large_straight",
    "full_house",
    "chance",
    "yatzy",
])

In [ ]:
categories

frozenset({'chance',
           'fives',
           'four_same',
           'fours',
           'full_house',
           'large_straight',
           'one_pair',
           'ones',
           'sixes',
           'small_straight',
           'three_same',
           'threes',
           'two_pairs',
           'twos',
           'yatzy'})

In [ ]:
# dice values
dice_vals = [5, 4, 3, 2, 1]
sorted_vals = tuple(sorted(dice_vals))
sorted_vals

(1, 2, 3, 4, 5)

In [ ]:
score = sum(sorted_vals)
score

15

In [ ]:
random.choices([1,2,3,4,5,6], k=5)

[3, 1, 6, 3, 4]

In [ ]:
#| export
def roll_n_dice(n):
    return random.choices([1,2,3,4,5,6], k=n)

In [ ]:
roll_n_dice(5)

[2, 1, 4, 2, 5]

## Planning more about how to represent state and actions

Okay, thinking about actions.

I would have 2 rolls left.

I can choose what dices to hold and which to re-roll. So basically 5 long binary word.

I can also choose small straight category, but no chance as its taken already.

I can also choose to set zero to any category that I've not taken already.

so action could be like
```
action = [00000, "small_straight", None]
# or
action = [11000, None, None]
# or
action = [00000, None, "ones"]
```

I need to have a function to compute possible actions.
* allow re-roll if "rolls left" is > 0
* check which categories can be selected
* which categories could be set to zero

## Back to implementing

In [ ]:
#| export
@dataclass
class State:
    dice_values: tuple[int, ...]
    rolls_left: int
    categories_picked: frozenset[str]

In [ ]:
state = State((1, 2, 3, 4, 5), 2, frozenset({'chance', 'large_straight', 'yatzy'}))
state

State(dice_values=(1, 2, 3, 4, 5), rolls_left=2, categories_picked=frozenset({'chance', 'large_straight', 'yatzy'}))

In [ ]:
#| export
@dataclass
class Action:
    type: Literal["reroll", "score"]
    value: tuple[int, ...] | str

In [ ]:
Action("reroll", (0,0,1,0,0))

Action(type='reroll', value=(0, 0, 1, 0, 0))

In [ ]:
Action("score", "ones")

Action(type='score', value='ones')

In [ ]:
#| export
reroll_masks = list(itertools.product((0,1), repeat=5))

In [ ]:
reroll_masks

[(0, 0, 0, 0, 0),
 (0, 0, 0, 0, 1),
 (0, 0, 0, 1, 0),
 (0, 0, 0, 1, 1),
 (0, 0, 1, 0, 0),
 (0, 0, 1, 0, 1),
 (0, 0, 1, 1, 0),
 (0, 0, 1, 1, 1),
 (0, 1, 0, 0, 0),
 (0, 1, 0, 0, 1),
 (0, 1, 0, 1, 0),
 (0, 1, 0, 1, 1),
 (0, 1, 1, 0, 0),
 (0, 1, 1, 0, 1),
 (0, 1, 1, 1, 0),
 (0, 1, 1, 1, 1),
 (1, 0, 0, 0, 0),
 (1, 0, 0, 0, 1),
 (1, 0, 0, 1, 0),
 (1, 0, 0, 1, 1),
 (1, 0, 1, 0, 0),
 (1, 0, 1, 0, 1),
 (1, 0, 1, 1, 0),
 (1, 0, 1, 1, 1),
 (1, 1, 0, 0, 0),
 (1, 1, 0, 0, 1),
 (1, 1, 0, 1, 0),
 (1, 1, 0, 1, 1),
 (1, 1, 1, 0, 0),
 (1, 1, 1, 0, 1),
 (1, 1, 1, 1, 0),
 (1, 1, 1, 1, 1)]

In [ ]:
available_actions = []
if state.rolls_left > 0:
    rerolls = [Action("reroll", mask) for mask in reroll_masks]
    available_actions.extend(rerolls)
available_actions

[Action(type='reroll', value=(0, 0, 0, 0, 0)),
 Action(type='reroll', value=(0, 0, 0, 0, 1)),
 Action(type='reroll', value=(0, 0, 0, 1, 0)),
 Action(type='reroll', value=(0, 0, 0, 1, 1)),
 Action(type='reroll', value=(0, 0, 1, 0, 0)),
 Action(type='reroll', value=(0, 0, 1, 0, 1)),
 Action(type='reroll', value=(0, 0, 1, 1, 0)),
 Action(type='reroll', value=(0, 0, 1, 1, 1)),
 Action(type='reroll', value=(0, 1, 0, 0, 0)),
 Action(type='reroll', value=(0, 1, 0, 0, 1)),
 Action(type='reroll', value=(0, 1, 0, 1, 0)),
 Action(type='reroll', value=(0, 1, 0, 1, 1)),
 Action(type='reroll', value=(0, 1, 1, 0, 0)),
 Action(type='reroll', value=(0, 1, 1, 0, 1)),
 Action(type='reroll', value=(0, 1, 1, 1, 0)),
 Action(type='reroll', value=(0, 1, 1, 1, 1)),
 Action(type='reroll', value=(1, 0, 0, 0, 0)),
 Action(type='reroll', value=(1, 0, 0, 0, 1)),
 Action(type='reroll', value=(1, 0, 0, 1, 0)),
 Action(type='reroll', value=(1, 0, 0, 1, 1)),
 Action(type='reroll', value=(1, 0, 1, 0, 0)),
 Action(type=

In [ ]:
can_select = categories - state.categories_picked
can_select

frozenset({'fives',
           'four_same',
           'fours',
           'full_house',
           'one_pair',
           'ones',
           'sixes',
           'small_straight',
           'three_same',
           'threes',
           'two_pairs',
           'twos'})

In [ ]:
#| export
def score_ns(n, values):
    return n * values.count(n)

assert score_ns(3, (1,2,3,4,5)) == 3
assert score_ns(2, (2,2,2,3,3)) == 6
assert score_ns(6, (1,2,3,4,5)) == 0

In [ ]:
#| export
def n_same(n, values):
    counts = Counter(values)
    return [k for k, v in counts.items() if v >= n]

assert n_same(2, (1,2,2,3,3)) == [2, 3]
assert n_same(2, (1,2,3,4,5)) == []

def score_n_same(n, values):
    sames = n_same(2, values)
    if len(sames) == 0: return 0
    return n * max(sames)

assert score_n_same(2, (3,3,4,4,6)) == 8
assert score_n_same(2, (1,2,3,4,6)) == 0

In [ ]:
#| export
def score_two_pairs(values):
    pairs = n_same(2, values)
    if len(pairs) == 2:
        return 2 * sum(pairs)
    return 0

assert score_two_pairs((1,1,2,2,3)) == 6
assert score_two_pairs((1,1,2,3,4)) == 0
assert score_two_pairs((1,1,1,1,2)) == 0

In [ ]:
#| export
def score_full_house(values):
    pairs = n_same(2, values)
    three_sames = n_same(3, values)
    if pairs and three_sames:
        pairs.remove(three_sames[0])
        if len(pairs) > 0: return sum(values)
    return 0

assert score_full_house((2,2,3,3,3)) == 13
assert score_full_house((1,2,3,3,3)) == 0

In [ ]:
#| export
def score_small_straight(values):
    return sum(values) if values == (1,2,3,4,5) else 0

def score_large_straight(values):
    return sum(values) if values == (2,3,4,5,6) else 0

assert score_small_straight((1,2,3,4,5)) == 15
assert score_large_straight((2,3,4,5,6)) == 20

In [ ]:
#| export
score_fn = {
    "ones": partial(score_ns, 1),
    "twos": partial(score_ns, 2),
    "threes": partial(score_ns, 3),
    "fours": partial(score_ns, 4),
    "fives": partial(score_ns, 5),
    "sixes": partial(score_ns, 6),
    "one_pair": partial(score_n_same, 2),
    "two_pairs": score_two_pairs,
    "three_same": partial(score_n_same, 3),
    "four_same": partial(score_n_same, 4),
    "small_straight": score_small_straight,
    "large_straight": score_large_straight,
    "full_house": score_full_house,
    "chance": lambda x: sum(x),
    "yatzy": partial(score_n_same, 5),
}

In [ ]:
for category in can_select:
    score = score_fn[category](state.dice_values)
    print(category, score)

four_same 0
three_same 0
threes 3
small_straight 15
fours 4
fives 5
sixes 0
ones 1
one_pair 0
two_pairs 0
full_house 0
twos 2


In [ ]:
available_actions

[Action(type='reroll', value=(0, 0, 0, 0, 0)),
 Action(type='reroll', value=(0, 0, 0, 0, 1)),
 Action(type='reroll', value=(0, 0, 0, 1, 0)),
 Action(type='reroll', value=(0, 0, 0, 1, 1)),
 Action(type='reroll', value=(0, 0, 1, 0, 0)),
 Action(type='reroll', value=(0, 0, 1, 0, 1)),
 Action(type='reroll', value=(0, 0, 1, 1, 0)),
 Action(type='reroll', value=(0, 0, 1, 1, 1)),
 Action(type='reroll', value=(0, 1, 0, 0, 0)),
 Action(type='reroll', value=(0, 1, 0, 0, 1)),
 Action(type='reroll', value=(0, 1, 0, 1, 0)),
 Action(type='reroll', value=(0, 1, 0, 1, 1)),
 Action(type='reroll', value=(0, 1, 1, 0, 0)),
 Action(type='reroll', value=(0, 1, 1, 0, 1)),
 Action(type='reroll', value=(0, 1, 1, 1, 0)),
 Action(type='reroll', value=(0, 1, 1, 1, 1)),
 Action(type='reroll', value=(1, 0, 0, 0, 0)),
 Action(type='reroll', value=(1, 0, 0, 0, 1)),
 Action(type='reroll', value=(1, 0, 0, 1, 0)),
 Action(type='reroll', value=(1, 0, 0, 1, 1)),
 Action(type='reroll', value=(1, 0, 1, 0, 0)),
 Action(type=

In [ ]:
for cat in can_select:
    score = score_fn[cat](state.dice_values)
    available_actions.append(Action(type="score", value=cat))
available_actions

[Action(type='reroll', value=(0, 0, 0, 0, 0)),
 Action(type='reroll', value=(0, 0, 0, 0, 1)),
 Action(type='reroll', value=(0, 0, 0, 1, 0)),
 Action(type='reroll', value=(0, 0, 0, 1, 1)),
 Action(type='reroll', value=(0, 0, 1, 0, 0)),
 Action(type='reroll', value=(0, 0, 1, 0, 1)),
 Action(type='reroll', value=(0, 0, 1, 1, 0)),
 Action(type='reroll', value=(0, 0, 1, 1, 1)),
 Action(type='reroll', value=(0, 1, 0, 0, 0)),
 Action(type='reroll', value=(0, 1, 0, 0, 1)),
 Action(type='reroll', value=(0, 1, 0, 1, 0)),
 Action(type='reroll', value=(0, 1, 0, 1, 1)),
 Action(type='reroll', value=(0, 1, 1, 0, 0)),
 Action(type='reroll', value=(0, 1, 1, 0, 1)),
 Action(type='reroll', value=(0, 1, 1, 1, 0)),
 Action(type='reroll', value=(0, 1, 1, 1, 1)),
 Action(type='reroll', value=(1, 0, 0, 0, 0)),
 Action(type='reroll', value=(1, 0, 0, 0, 1)),
 Action(type='reroll', value=(1, 0, 0, 1, 0)),
 Action(type='reroll', value=(1, 0, 0, 1, 1)),
 Action(type='reroll', value=(1, 0, 1, 0, 0)),
 Action(type=

In [ ]:
#| export
def get_available_actions(state: State) -> list[Action]:
    available_actions = []
    if state.rolls_left > 0:
        rerolls = [Action("reroll", mask) for mask in reroll_masks]
        available_actions.extend(rerolls)
    
    can_select = categories - state.categories_picked
    for cat in can_select:
        score = score_fn[cat](state.dice_values)
        if can_select:
            available_actions.append(Action(type="score", value=cat))
    
    return available_actions

In [ ]:
get_available_actions(state)

[Action(type='reroll', value=(0, 0, 0, 0, 0)),
 Action(type='reroll', value=(0, 0, 0, 0, 1)),
 Action(type='reroll', value=(0, 0, 0, 1, 0)),
 Action(type='reroll', value=(0, 0, 0, 1, 1)),
 Action(type='reroll', value=(0, 0, 1, 0, 0)),
 Action(type='reroll', value=(0, 0, 1, 0, 1)),
 Action(type='reroll', value=(0, 0, 1, 1, 0)),
 Action(type='reroll', value=(0, 0, 1, 1, 1)),
 Action(type='reroll', value=(0, 1, 0, 0, 0)),
 Action(type='reroll', value=(0, 1, 0, 0, 1)),
 Action(type='reroll', value=(0, 1, 0, 1, 0)),
 Action(type='reroll', value=(0, 1, 0, 1, 1)),
 Action(type='reroll', value=(0, 1, 1, 0, 0)),
 Action(type='reroll', value=(0, 1, 1, 0, 1)),
 Action(type='reroll', value=(0, 1, 1, 1, 0)),
 Action(type='reroll', value=(0, 1, 1, 1, 1)),
 Action(type='reroll', value=(1, 0, 0, 0, 0)),
 Action(type='reroll', value=(1, 0, 0, 0, 1)),
 Action(type='reroll', value=(1, 0, 0, 1, 0)),
 Action(type='reroll', value=(1, 0, 0, 1, 1)),
 Action(type='reroll', value=(1, 0, 1, 0, 0)),
 Action(type=

**What I have**

* state representation
* action representation
* categories
* dice rolling
* functions to check if dice values fit category
* available actions function

**What is missing**
* transition function
    * logic for reroll
    * logic for selecting zeroing categories

**Transition function**

state + action --> next state

```python
next_state = transition_func(state, action)
```

**What transitions are there?**

Action types:
* reroll
* zero
* score

**reroll**
* reroll the dice according to mask
* set new dice to state
* decrement rolls left

**zero**
* set the category as selected
* check wheter categories left and set terminal state if not any left
* reroll whole dice
* reset rolls left to 2

**score**
* compute score
* set the category as selected
* add score to scoreboard or total sum
* check wheter categories left and set terminal state if not any left
* reroll whole dice
* reset rolls left to 2


Let's have a state and action to work with.

In [ ]:
state

State(dice_values=(1, 2, 3, 4, 5), rolls_left=2, categories_picked=frozenset({'chance', 'large_straight', 'yatzy'}))

In [ ]:
action = Action(type='reroll', value=(1, 1, 1, 0, 0))
action

Action(type='reroll', value=(1, 1, 1, 0, 0))

What it would mean to transition to next state given this state and action?

In [ ]:
#| export
def reroll_masked(values, mask):
    next_values = list(values)
    reroll = roll_n_dice(5)
    for i, bit in enumerate(mask):
        if bit == 1:
            next_values[i] = reroll[i]
    return tuple(sorted(next_values))

In [ ]:
reroll_masked((1,2,3,4,5), (1,1,1,0,0))

(4, 4, 5, 5, 6)

In [ ]:
#| export
def transition_func(state: State, action: Action):
    match action.type:
        case "reroll":
            assert state.rolls_left > 0
            new_values = reroll_masked(state.dice_values, action.value)
            return State(new_values, state.rolls_left - 1, state.categories_picked)
        case "score":
            assert action.value not in state.categories_picked
            new_categories = state.categories_picked | {action.value}
            categories_left = categories - state.categories_picked
            # TODO: scoreboard stuff
            if not categories_left: return TERMINAL_STATE
            new_values = tuple(sorted(roll_n_dice(5)))
            return State(new_values, 2, frozenset(new_categories))


In [ ]:
transition_func(state, action)

State(dice_values=(1, 2, 3, 4, 5), rolls_left=1, categories_picked=frozenset({'chance', 'large_straight', 'yatzy'}))

In [ ]:
transition_func(state, Action("score", "ones"))

State(dice_values=(1, 2, 2, 6, 6), rolls_left=2, categories_picked=frozenset({'chance', 'ones', 'large_straight', 'yatzy'}))

In [ ]:
!cat yatzy/yatzy.py

# AUTOGENERATED! DO NOT EDIT! File to edit: ../nbs/yatzy_dup1.ipynb.

# %% auto 0
__all__ = ['categories', 'reroll_masks', 'checkers', 'roll_n_dice', 'State', 'Action', 'has_ns', 'n_same', 'has_n_same',
           'has_two_pairs', 'has_full_house', 'has_small_straight', 'has_large_straight', 'get_available_actions',
           'reroll_masked', 'transition_func']

# %% ../nbs/yatzy_dup1.ipynb 5
import random
from dataclasses import dataclass
from typing import Literal
import itertools
from collections import Counter
from functools import partial

# %% ../nbs/yatzy_dup1.ipynb 6
# categories

categories = frozenset([
    "ones",
    "twos",
    "threes",
    "fours",
    "fives",
    "sixes",
    "one_pair",
    "two_pairs",
    "three_same",
    "four_same",
    "small_straight",
    "large_straight",
    "full_house",
    "chance",
    "yatzy",
])

# %% ../nbs/yatzy_dup1.ipynb 11
def roll_n_dice(n):
    return random.choices([1,2,3,4,5,6], k=n)

# %% ../nbs/yatzy_dup1.ipynb 16
@dataclas